<p dir="rtl"> گوگل درایو را مانت میکنیم. از درایو برای ذخیره‌سازی عکس‌های پردازش شده استفاده می‌کنیم. </p>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<p dir="rtl"> برای دانلود تصاویر پاتولوژی نیاز است GDC Client نصب شود. سعی می‌کنیم این پکیج را build کنیم. </p>

In [ ]:
!git clone https://github.com/NCI-GDC/gdc-client.git
%cd /content/gdc-client
!pip install -r requirements.txt

Cloning into 'gdc-client'...
remote: Enumerating objects: 2631, done.
remote: Counting objects: 100% (315/315), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 2631 (delta 207), reused 259 (delta 191), pack-reused 2316
Receiving objects: 100% (2631/2631), 6.12 MiB | 31.81 MiB/s, done.
Resolving deltas: 100% (1618/1618), done.
/content/gdc-client
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.2/149.2 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 KB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 KB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 83.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.6/72.6 KB 9.5 MB/s eta 0:00:00
     ━━━━

In [ ]:
%cd /content/gdc-client
!python setup.py install 2>&1 | tee -a install.log

/content/gdc-client
running install
running bdist_egg
running egg_info
creating gdc_client.egg-info
writing gdc_client.egg-info/PKG-INFO
writing dependency_links to gdc_client.egg-info/dependency_links.txt
writing requirements to gdc_client.egg-info/requires.txt
writing top-level names to gdc_client.egg-info/top_level.txt
writing manifest file 'gdc_client.egg-info/SOURCES.txt'
adding license file 'LICENSE'
adding license file 'NOTICE.txt'
writing manifest file 'gdc_client.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/gdc_client
copying gdc_client/__init__.py -> build/lib/gdc_client
copying gdc_client/exceptions.py -> build/lib/gdc_client
copying gdc_client/defaults.py -> build/lib/gdc_client
copying gdc_client/version.py -> build/lib/gdc_client
copying gdc_client/utils.py -> build/lib/gdc_client
creating build/lib/gdc_client/query
copying gdc_client/query/__init__.p

In [ ]:
%cd ..

/content


<p dir="rtl"> برای نرمال‌سازی تصاویر پاتولوژی نیاز است پکیج staintools نصب شود. </p>

In [ ]:
!pip install spams
!pip install staintools

<p dir="rtl"> پکیج openslide برای خواندن فایل با فرمت svs کاربرد دارد. تصاویر دانلود شده دارای فرمت svs هستند. </p>

In [ ]:
!apt update && apt install -y openslide-tools
!pip install openslide-python

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:14 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu 

In [ ]:
import os
import cv2
from PIL import Image
import staintools
from openslide import OpenSlide
from tqdm import tqdm
import numpy as np

<p dir="rtl"> برای دانلود تصاویر نیاز است که uuid آن را به gdc client پاس بدهیم. </p>

In [ ]:
def download_svs(UUID):
  os.system("gdc-client download " + UUID)  
  for file in os.listdir(UUID):
    if file.endswith(".svs"):
      return UUID + "/" + file

<p dir="rtl"> هر عکس را به پچ‌های 1024 و 2048 و 3072 پیکسلی تقسیم میکنیم. سپس این پچ‌ها را تبدیل به یک عکس 299 در 299 میکنیم. در این‌جا سعی شده است با الگوریتم otsu پچ‌هایی که حاوی اطلاعات مهمی نیستند حذف شوند. </p>

In [ ]:
def iterate(UUID, source_image, w, h, normalizer, patch_size, output_shape):
  global UUID_to_filename
  for x in tqdm(range(500, w - 500, patch_size)):
    if x + patch_size >= w - 510:
      continue
    for y in range(500, h - 500, patch_size):
      if y + patch_size >= h - 510:
        continue
      patched_image = source_image.read_region(location=(x, y), level=0, size=(patch_size, patch_size)).convert("RGB").resize(output_shape, Image.LANCZOS)
      patched_image = np.array(patched_image)
      gray_image = cv2.cvtColor(patched_image, cv2.COLOR_BGR2GRAY)
      ret, th = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
      white_percentage1 = np.where(gray_image > 220, 1, 0).sum() / (output_shape[0] * output_shape[0])
      white_percentage2 = np.count_nonzero(th[th == 255]) / (output_shape[0] * output_shape[0])
      if white_percentage2 < 0.9 and white_percentage1 < 0.9:
        try:
          normalized_image = normalizer.transform(patched_image)
        except:
          continue
        I = Image.fromarray(patched_image)
        path = "/content/drive/MyDrive/histo/" + UUID_to_filename[UUID] + "/" + str(patch_size) + "/" + str(x) + "_" + str(y) + ".png"
        I.save(path, quality=95)

def patch(UUID, target_image_path, source_image_path):
    global UUID_to_filename
    main_path = "/content/drive/MyDrive/histo/"
    if not os.path.exists(main_path + UUID_to_filename[UUID]):
      os.mkdir(main_path + UUID_to_filename[UUID])

    if not os.path.exists(main_path + UUID_to_filename[UUID] + "/" + "1024"):
      os.mkdir(main_path + UUID_to_filename[UUID] + "/" + "1024")
    if not os.path.exists(main_path + UUID_to_filename[UUID] + "/" + "2048"):
      os.mkdir(main_path + UUID_to_filename[UUID] + "/" + "2048")
    if not os.path.exists(main_path + UUID_to_filename[UUID] + "/" + "3072"):
      os.mkdir(main_path + UUID_to_filename[UUID] + "/" + "3072")

    source_image = OpenSlide(source_image_path)
    normalizer = staintools.StainNormalizer(method='vahadane')
    target_image = staintools.read_image(target_image_path)
    normalizer.fit(target_image)
    # normalizer = None

    w, h = source_image.dimensions
    output_shape = (299, 299)

    iterate(UUID, source_image, w, h, normalizer, 1024, output_shape)
    iterate(UUID, source_image, w, h, normalizer, 2048, output_shape)
    iterate(UUID, source_image, w, h, normalizer, 3072, output_shape)

<p dir="rtl"> uuid ها را از روی فایل manifest پیدا میکنیم و تک به تک پردازش‌های گفته‌شده در بالا را روی آن‌ها اجرا می‌کنیم. </p>

In [ ]:
UUID_to_filename = {}
with open('manifest.txt', 'r') as file:
  for line in file:
    break
  for line in file:
    UUID, filename, md5, size, status = line.split()
    UUID_to_filename[UUID] = filename.split(".")[0]
    size = int(size) // 1024 ** 2
    if size > 500:
      continue
    if not os.path.exists("/content/drive/MyDrive/histo/" + UUID_to_filename[UUID]):
      print(f'downloading {filename} ... with size {size} MB')
      filepath = download_svs(UUID)
      print("download ended")
      patch(UUID, "colorstandard_brca.png", filepath)
      os.remove(filepath)
    else:
      pass
      # print("using existing file...", filename)
      # for sub_file in os.listdir(UUID):
      #   if sub_file.endswith(".svs"):
      #     filepath =  UUID + "/" + sub_file
      #     break


downloading TCGA-A7-A0CJ-01Z-00-DX1.E26F2F62-D688-4373-BB7B-790A06734E49.svs ... with size 220 MB
download ended


100%|██████████| 10/10 [02:35<00:00, 15.57s/it]


downloading TCGA-A2-A0YE-01A-01-TS1.03c71807-b82c-4b01-8962-c6ac3d1b3445.svs ... with size 384 MB
download ended


100%|██████████| 38/38 [04:32<00:00,  7.16s/it]


downloading TCGA-BH-A0BO-11A-01-TSA.dd5e8987-6e95-4412-b76a-c36cadb44da9.svs ... with size 323 MB
download ended


...

100%|██████████| 54/54 [07:06<00:00,  7.90s/it]


downloading TCGA-BH-A5J0-01A-01-TS1.0088B960-4EA8-4BD3-B9B4-7C1FE59D4925.svs ... with size 264 MB
download ended


100%|██████████| 36/36 [04:09<00:00,  6.94s/it]


downloading TCGA-A7-A0CJ-01Z-00-DX2.4B591117-5FC9-4B43-8A45-444CCCABC666.svs ... with size 253 MB
download ended


100%|██████████| 13/13 [03:19<00:00, 15.31s/it]
